# Search Project for CST 495

> CMU Movie Summary Corpus
http://www.cs.cmu.edu/~ark/personas/

We will be using Spark, so the first step is to ensure we have installed the module.

In [2]:
! pip install findspark

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# First we specify the path to spark
import findspark
import os
findspark.init(os.getenv('HOME') + '/spark-1.6.0-bin-hadoop2.6')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'

# Now we can import pyspark and get the spark context
# - spark context is the entry point to spark for a spark application
import pyspark
try: 
    print(sc)
except NameError:
    sc = pyspark.SparkContext()
    print(sc)

IndexError: list index out of range

# Resilient Distributed Dataset (RDD)

From the Spark documentation:

_"A Resilient Distributed Dataset (RDD), the basic abstraction in Spark, represents an immutable, partitioned collection of elements that can be operated on in parallel."_

_"Parallelized collections are created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program. The elements of the collection are copied to form a distributed dataset that can be operated on in parallel."_ 

In [ ]:
# creating an RDD

rdd = sc.textFile('data/MovieSummaries/plot_summaries.txt')
print(rdd)

rdd.take(3)

> **Counting words**

In [ ]:
words_per_line = rdd.map(lambda s: len(s.split())).filter(lambda x : x > 2)

total_words = words_per_line.reduce(lambda x,y : x+y)

print(total_words)

> Term Frequency

In [ ]:
# use Spark API for optimization
import re

def normalizeWords(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())
    #return re.compile(r'\b[a-zA-Z]+\b').split(text.lower())

def toCSVLine(data):
    return ','.join(str(d) for d in data)

rdd = sc.textFile(os.getcwd()+'/data/MovieSummaries/plot_summaries.txt')
rdd.cache()
#words = rdd.flatMap(lambda x: x.split())
words = rdd.flatMap(normalizeWords)
wordCounts = words.countByValue()

wordCounts = words.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda (x,y) : (y,x)).sortByKey()
results = wordCountsSorted.collect()

import csv
with open(os.getcwd() + '/data/MovieSummaries/plot_sum.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ')#,
                            #quotechar=' ', quoting=csv.QUOTE_MINIMAL)

    for result in results:
        count = str(result[0])
        word = result[1].encode('ascii', 'ignore')
        
        if(word and int(count)>10000): # (word.isdigit()):# and int(count)<2):
                print word + ":\t\t" + count

                #limit csv filr for now
                spamwriter.writerow([count] + [","] + [word])
          

         
            

> Testing the Spark DataFrames API with the Data

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', inferSchema='true').load(os.getcwd() 
        + '/data/MovieSummaries/plot_sum.csv').selectExpr("C0 as id","C1 as words")

df.show()

In [ ]:
df.schema

In [ ]:
sqlContext.registerDataFrameAsTable(df,'plotTerms')
sqlContext.tableNames()

sqlContext.sql("select * from plotTerms order by id limit 20").show()

In [ ]:
sqlContext.tableNames()

In [ ]:
df.take(10)

> **Inverted Index**

In [ ]:
index = df.flatMap(lambda row : [ ( word, row[0]) for word in row[1].strip().split(' ') ] ) 
index.take(20)

In [ ]:
index = df.flatMap(lambda row : [ (word,  row[0]) for word in row[1].split(' ') ] ).groupByKey()
index.take(10)

In [ ]:
index = df.flatMap(lambda row : [ (word,  row[0]) for word in row[1].split(' ') ] ).groupByKey().map(lambda x : (x[0], list(x[1])))
index.filter(lambda x : x[0] == 'father').collect()

In [ ]:
index = df.flatMap(lambda row : [ (word,  row[0]) for word in row[1].split(' ') ] ).groupByKey().map(lambda x : (x[0], list(x[1])))
index.filter(lambda x : x[0] == 'mother').collect()

# Spark DataFrames API

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
mov_df = sqlContext.read.format('com.databricks.spark.csv').options(delimiter='\t', header='false', inferSchema='true').load(os.getcwd() 
        + '/data/MovieSummaries/movie.metadata.tsv').selectExpr("C0 as wiki_id","C2 as movie_title", "C3 as release_date", "C4 as box_office_rev", "C5 as runtime"
                                                               ,"C6 as languages", "C7 as countries")

mov_df.show()

In [ ]:
plot_df = sqlContext.read.format('com.databricks.spark.csv').options(delimiter="\t", header='false', inferSchema='true').load(os.getcwd() 
        + '/data/MovieSummaries/plot_summaries.txt').selectExpr("C0 as wiki_id", "C1 as plot")

plot_df.show()

In [ ]:
# turn movie data frame into table
sqlContext.registerDataFrameAsTable(mov_df,'movieMeta')
sqlContext.tableNames()

sqlContext.sql("select * from movieMeta order by release_date desc limit 20").show()

In [ ]:
sqlContext.registerDataFrameAsTable(plot_df,'plotTerms')
sqlContext.tableNames()

sqlContext.sql("select * from plotTerms order by wiki_id limit 20").show()

In [ ]:
sqlContext.sql("select * from movieMeta where wiki_id >4725 and wiki_id <4731 limit 20").show()

In [ ]:
new_df = sqlContext.sql("select movie_title, words from movieMeta left outer join plotTerms")

sqlContext.registerDataFrameAsTable(new_df,'titleWord')
sqlContext.tableNames()

In [ ]:
rdd = plot_df.rdd

rdd.take(1)

In [ ]:
index = rdd.flatMap(lambda row : [ ( word, row[0]) for word in row[1].split(' ') ] ) 
index.take(50)

In [ ]:
index = rdd.flatMap(lambda row : [ (word,  row[0]) for word in row[1].split(' ') ] ) \
            .groupByKey()
index.take(10)

In [ ]:
index = rdd.flatMap(lambda row : [ (word,  row[0]) for word in row[1].split(' ') ] ) \
            .groupByKey() \
            .map(lambda x : (x[0], list(x[1]))).cache()

In [ ]:
indices = index.filter(lambda x : x[0] == 'green').take(10)
tup = tuple(indices[0][1])

In [ ]:
sqlContext.sql("select movie_title from movieMeta where wiki_id in " + str(tup) + " order by wiki_id ").show()

# Let's try this again

>this time we will bring it back to the basics. We will normalise the text by removing unwanted characters and converting to lowercase

In [4]:
import csv
import re

with open("data/MovieSummaries/plot_summaries.tsv") as f:
    r = csv.reader(f, delimiter='\t', quotechar='"')
    tag = re.compile(r'\b[0-9]+\b')
    rgx = re.compile(r'\b[a-zA-Z]+\b')
    #docs = [ (' '.join(re.findall(tag, x[0])).lower(), ' '.join(re.findall(rgx, x[1])).lower()) for i,x in enumerate(r) if r>1 ]
    docs= {}
    for i,x in enumerate(r):
        if i >1:
            docs[' '.join(re.findall(tag, x[0])).lower()] = ' '.join(re.findall(rgx, x[1])).lower()
#print(docs[0][0], docs[0][1])    

#item_t = [ d[0] for d in docs ] # item titles
#tem_d = [ d[1] for d in docs ] # item description
#item_i = range(0 , len(item_t)) # item id





> now to normalize the movie meta data to swap the item titles with index from above

** just the basics for now to get index, possibility to get genre if needed **

In [5]:
import csv
import re

with open("data/MovieSummaries/movie.metadata.tsv") as f:
    r = csv.reader(f, delimiter='\t', quotechar='"')
    tag = re.compile(r'\b[0-9]+\b')
    rgx = re.compile(r'\b[a-zA-Z]+\b')
    #docs2 = [ (' '.join(re.findall(tag, x[0])).lower(), ' '.join(re.findall(rgx, x[2])).lower()) for i,x in enumerate(r) if r>1 ]
    docs2= {}
    for i,x in enumerate(r):
        if i >1:
            docs2[' '.join(re.findall(tag, x[0])).lower()] = ' '.join(re.findall(rgx, x[2])).lower()
            
#print(docs2)

> now is the time to join the docs together

In [6]:
doc = [(docs2.get(x), y) for x, y in docs.items() if docs2.get(x)]



# for testing
# import random
 #print doc[random.randint(0, len(doc)-1)]
print doc[0][0], doc[0][1]

items_t = [ d[0] for d in doc ] # item titles
items_d = [ d[1] for d in doc ] # item description
items_i = range(0 , len(items_t)) # item id



periya idathu penn murugappa is a small time farm labourer who lives with his widowed sister gangamma in a village pillaival is the zamindar of the village and sabapathy and punitha are his children punitha is studying in college in a nearby town while sabapathy is not educated both the father and the children are both arrogant about their wealth and try to rule the villagers murugappa tries to question their authority and this leads to frequent clashes with the zamindar s family pichandi is a wealthy college mate of punitha who is crazy about her sabapathy falls in love with thillaiammal who has been informally enagaged to murugappa for a long time both pillaival and gangamma propose for her on the same day to avoid a direct clash with the zamindar her father says that he took a vow that his daughter would marry the winner of a silambam competition punitha promises to marry pichandi if he dopes a drink which murugappa drinks during the fight sabapathy wins the fight and marries thilak

# term freq

In [50]:
corpus = items_d[0:25]
print corpus

['murugappa is a small time farm labourer who lives with his widowed sister gangamma in a village pillaival is the zamindar of the village and sabapathy and punitha are his children punitha is studying in college in a nearby town while sabapathy is not educated both the father and the children are both arrogant about their wealth and try to rule the villagers murugappa tries to question their authority and this leads to frequent clashes with the zamindar s family pichandi is a wealthy college mate of punitha who is crazy about her sabapathy falls in love with thillaiammal who has been informally enagaged to murugappa for a long time both pillaival and gangamma propose for her on the same day to avoid a direct clash with the zamindar her father says that he took a vow that his daughter would marry the winner of a silambam competition punitha promises to marry pichandi if he dopes a drink which murugappa drinks during the fight sabapathy wins the fight and marries thilakam punitha goes b

>> start by computing frequncy of entire corpus

In [8]:
tf = {}
for doc in corpus:
    for word in doc.split():
        if word in tf:
            tf[word] += 1
        else:
            tf[word] = 1
print(tf)

{'tomboy': 1, 'daybreak': 1, 'bicycle': 1, 'doll': 1, 'highlander': 1, 'dead': 1, 'ii': 1, 'salaakhen': 1, 'gang': 1, 'in': 1, 'colleen': 1, 'flock': 1, 'obsession': 1, 'slackistan': 1, 'to': 1, 'parvarish': 1, 'frontier': 1, 'odd': 1, 'acts': 1, 'three': 1, 'rivals': 1, 'secret': 1, 'scissors': 1, 'penn': 1, 'rider': 1, 'sergeant': 1, 'murder': 1, 'return': 1, 'thambikottai': 1, 'flying': 1, 'idathu': 1, 'vengeance': 1, 'gori': 1, 'rudy': 1, 'steel': 1, 'on': 1, 'great': 1, 'home': 1, 'knight': 1, 'mwansa': 1, 'drake': 1, 'bride': 1, 's': 1, 'ryker': 1, 'venture': 1, 'the': 4, 'relentless': 1, 'periya': 1}


>> now that we have normailised the data we can compute the term frequency


In [9]:
from collections import Counter

def get_tf(corpus):
    tf = Counter()
    for doc in corpus:
        for word in doc.split():
            tf[word] += 1
    return tf

tf = get_tf(corpus)
print(tf)
    

Counter({'the': 4, 'tomboy': 1, 'daybreak': 1, 'bicycle': 1, 'doll': 1, 'highlander': 1, 'dead': 1, 'ii': 1, 'salaakhen': 1, 'gang': 1, 'in': 1, 'colleen': 1, 'flock': 1, 'obsession': 1, 'slackistan': 1, 'to': 1, 'parvarish': 1, 'frontier': 1, 'odd': 1, 'acts': 1, 'three': 1, 'rivals': 1, 'secret': 1, 'scissors': 1, 'penn': 1, 'rider': 1, 'sergeant': 1, 'murder': 1, 'return': 1, 'thambikottai': 1, 'flying': 1, 'idathu': 1, 'vengeance': 1, 'gori': 1, 'rudy': 1, 'steel': 1, 'on': 1, 'great': 1, 'home': 1, 'knight': 1, 'mwansa': 1, 'drake': 1, 'bride': 1, 's': 1, 'ryker': 1, 'venture': 1, 'relentless': 1, 'periya': 1})


# doc freq
> 

In [10]:
import collections

def get_tf(document):
    tf = Counter()
    for word in document.split():
        tf[word] += 1
    return tf

def get_dtf(corpus):
    dtf = {}
    for i,doc in enumerate(corpus):
        dtf[i]= get_tf(doc)
    return dtf

dtf = get_dtf(items_t)
dtf[342]

Counter({'and': 1, 'give': 1, 'tyke': 1})

> compute dtf for item descriptions

In [11]:
dtf = get_dtf(items_d)
dtf[12]

Counter({'a': 10,
         'ability': 1,
         'accept': 1,
         'affection': 1,
         'after': 1,
         'against': 1,
         'an': 1,
         'and': 6,
         'are': 1,
         'aristocrat': 1,
         'aristocratic': 1,
         'aristocrats': 2,
         'army': 1,
         'as': 3,
         'at': 3,
         'aware': 1,
         'bankrupt': 1,
         'because': 1,
         'becomes': 2,
         'begins': 1,
         'bulgaria': 1,
         'business': 2,
         'but': 3,
         'cinema': 1,
         'cki': 1,
         'com': 1,
         'comes': 1,
         'company': 1,
         'condescended': 1,
         'consents': 1,
         'database': 1,
         'daughter': 1,
         'descendant': 1,
         'devotion': 1,
         'distressed': 1,
         'dreaming': 1,
         'during': 1,
         'edu': 1,
         'end': 1,
         'enterprising': 1,
         'eventual': 1,
         'exile': 2,
         'failed': 1,
         'falling': 1,
         'fam

# term freq matrix

In [12]:
def get_lexicon(corpus):
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc.split()])
    return list(lexicon)

test_corpus = ['mountain bike', 'road bike carbon', 'bike helmet']
lexicon = get_lexicon(test_corpus)
print lexicon

['mountain', 'helmet', 'bike', 'road', 'carbon']


> with the lexicon we are able to compute the term freq matrix

In [13]:
def get_tfm(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return list(lexicon)
    
    lexicon = get_lexicon(corpus)
    
    tfm =[]
    for doc in corpus:
        tfv = [0]*len(lexicon)
        for term in doc.split():
            tfv[lexicon.index(term)] += 1
    
        tfm.append(tfv)
    
    return tfm, lexicon

test_corpus = ['mountain bike', 'road bike carbon', 'bike helmet']
tfm, lexicon = get_tfm(test_corpus)
print lexicon
print tfm


    

['mountain', 'helmet', 'bike', 'road', 'carbon']
[[1, 0, 1, 0, 0], [0, 0, 1, 1, 1], [0, 1, 1, 0, 0]]


# sparsity of term frequency matrix

In [14]:
! pip install bokeh

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show, vplot

# sparsity as a function of document count
n = []
s = []
for i in range(100,1000,100):
    corpus = items_t[0:i]
    tfm, lexicon = get_tfm(corpus)
    c = [ [x.count(0), x.count(1)] for x in tfm]
    n_zero = sum([ y[0] for y in c])
    n_one = sum( [y[1] for y in c])
    s.append(1.0 - (float(n_one) / (n_one + n_zero)))
    n.append(i)
    
output_notebook(hide_banner=True)
p = figure(x_axis_label='Documents', y_axis_label='Sparsity', plot_width=400, plot_height=400)
p.line(n, s, line_width=2)
p.circle(n, s, fill_color="white", size=8)
show(p)

# boolean search


We are now in a position to write our first ranking function.  Now we have the term frequency matrix we can use it to find documents that contain words included in a user specified query.  We will start by simply returning the documents from the corpus that match any terms in the query and rank by the raw frequency of matching terms. 

More specifically our algorithm for 'boolean search' proceeds as follows:

* Compute the lexicon for the corpus
* Compute the term frequency matrix for the corpus
* Convert query to query vector using the same lexicon 
* Compare each documents term frequncy vector to the query vector - specifically for each document in the corpus:
    * Compute a ranking score for each document by taking the [dot product](https://en.wikipedia.org/wiki/Dot_product) of the document's term frequency vector and the query vector
* Sort the documents by ranking score

In [16]:
#test_corpus = ['mountain bike red', 'road bike carbon', 'bike helmet']
#print test_corpus

# compute term frequency matrix and lexicon
tfm, lexicon = get_tfm(corpus)

#print tfm
#print lexicon

# define our query
qry = 'red bike'

# convert query to query vector using lexicon
qrv = [0]*len(lexicon)
for term in qry.split():
    if term in lexicon:
        qrv[lexicon.index(term)] = 1

#print qrv

# compare query vector to each term frequency vector
# this is dot product between qrv and each row of tfm
for i,tfv in enumerate(tfm):
    print i, sum([ xy[0] * xy[1] for xy in zip(qrv, tfv) ])

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 0
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 0
166 0
167 0
168 0
169 0
170 0
171 0
172 0
173 0
174 0
175 0
176 0
177 0
178 0
179 0
180 0
181 0
182 0
183 0
184 0


The function definition get_results_tf() computes the document ranking score for each document in the term frequency matrix

In [23]:
def get_results_tf(qry, tfm, lexicon):
    qrv =[0]*len(lexicon)
    for term in qry.split():
        if term in lexicon:
            qrv[lexicon.index(term)] = 1
            
    results = []
    for i, tfv in enumerate(tfm):
        score = 0
        score = sum([ xy[0] * xy[1] for xy in zip(qrv,tfv)])
        results.append([score, i])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1)
    return sorted_results


def print_results(results,n, head=True):
    ''' Helper function to print results
    '''
    if head:    
        print('\nTop %d from recall set of %d items:' % (n,len(results)))
        for r in results[:n]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    else:
        print('\nBottom %d from recall set of %d items:' % (n,len(results)))
        for r in results[-n:]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    

tfm, lexicon = get_tfm(items_d[:1000])
results = get_results_tf('fun times', tfm , lexicon)
print_results(results,10)


Top 10 from recall set of 1000 items:
	4.00 - the challenge
	3.00 - color me kubrick
	3.00 - halloween years later
	3.00 - b b s kids
	2.00 - the last day of summer
	2.00 - eti
	2.00 - halloweentown
	2.00 - des pissenlits par la racine
	2.00 - santouri
	2.00 - banjo the woodpile cat


# Inverted Index

> the inverted index maps terms to the document in which they can be found

In [24]:
def create_inverted_index(corpus):
    idx={}
    for i, document in enumerate(corpus):
        for word in document.split():
            if word in idx:
                idx[word].append(i)
            else:
                idx[word] = [i]
        ## HIDE
    return idx

test_corpus = ['mountain bike red','road bike carbon','bike helmet']
idx = create_inverted_index(test_corpus)
print(idx)

{'mountain': [0], 'helmet': [2], 'bike': [0, 1, 2], 'red': [0], 'carbon': [1], 'road': [1]}


> inverted index for document titles

In [29]:
idx = create_inverted_index(items_t)
print(set(idx['good']).intersection(set(idx['times'])))
print(items_t[2061])

set([22649])
bi the way


> improve the ranking function

In [33]:
def get_results_tf(qry, idx):
    score = Counter()
    for term in qry.split():
        for doc in idx[term]:
            score[doc] += 1
            
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results;


idx = create_inverted_index(items_d)
results = get_results_tf('zombies', idx)
print_results(results,20)


Top 20 from recall set of 190 items:
	30.00 - burial ground the nights of terror
	19.00 - dance of the dead
	19.00 - video dead
	16.00 - zombies zombies zombies
	14.00 - flesheater
	14.00 - big tits zombie
	13.00 - shaun of the dead
	12.00 - dead and deader
	12.00 - dawn of the dead
	11.00 - route
	11.00 - hide and creep
	11.00 - undead or alive
	10.00 - the stink of flesh
	10.00 - abraham lincoln vs zombies
	9.00 - night of the living dead
	9.00 - planet terror
	9.00 - when good ghouls go bad
	8.00 - zombi
	8.00 - day of the dead contagium
	8.00 - day of the dead


> enter different queries

In [35]:
results = get_results_tf('ghouls and ghosts', idx)
print_results(results, 10)


Top 10 from recall set of 39657 items:
	181.00 - in the line of duty witness
	165.00 - dragon head
	165.00 - band of the hand
	162.00 - underworld rise of the lycans
	145.00 - franklin and the green knight
	144.00 - devil s diary
	140.00 - wishology
	139.00 - the runaways
	134.00 - the guard post
	129.00 - the mists of avalon


In [38]:
import pandas as pd
from bokeh.plotting import output_notebook, show
from bokeh.charts import Bar
from bokeh.charts.attributes import CatAttr
#from bokeh.models import ColumnDataSource

df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()]})

output_notebook(hide_banner=True)
p = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
show(p)


# TF-IDF

$$
IDF = log ( 1 + \frac{N}{n_t} ) 
$$

In [43]:
import math

def idf(term, idx, n):
    return math.log( float(n) / (1 + len(idx[term])))    


print(idf('zombie',idx,len(items_d)))
print(idf('survival',idx,len(items_d)))
print(idf('invasions',idx,len(items_d)))

4.34894878018
4.91779824406
8.4506734497


### TF-IDF Intuition

In [44]:
from bokeh.charts import vplot

idx = create_inverted_index(items_d)

df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()],
                  'idf':[idf(x, idx, len(items_t)) for x in idx.keys()]})

output_notebook(hide_banner=True)
p1 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
p2 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='idf',
        plot_width=800, plot_height=400)
p = vplot(p1, p2)
show(p)

/Users/dustin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: BokehDeprecationWarning: bokeh.io.vplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Column instead


### TD-IDF Ranking 


In [52]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            if word in idx:
                if i in idx[word]:
                    # Update document's frequency
                    idx[word][i] += 1
                else:
                    # Add document
                    idx[word][i] = 1
            else:
                # Add term
                idx[word] = {i:1}
    return idx

def get_results_tfidf(qry, idx, n):
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                score[doc] += idx[term][doc] * i
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

idx = create_inverted_index(items_t)
results = get_results_tfidf('lookout action bike zombie', idx, len(items_d))
print_results(results,10)


Top 10 from recall set of 56 items:
	9.95 - the lookout
	9.26 - the kid with a bike
	9.26 - the bike
	9.26 - the dirt bike kid
	7.51 - delayed action
	7.51 - class action
	7.51 - action replayy
	7.51 - stand by for action
	7.51 - missing in action
	7.51 - looney tunes back in action


Ideally we do not want scores to be the same for lots of documents. High TF-IDF scores in shorter documents should be more relevant - so we could try by boosting the score for documents that are shorter than average.

In [53]:
def get_results_tfidf_boost(qry, corpus):
    idx = create_inverted_index(corpus)
    n = len(corpus)
    d = [len(x.split()) for x in corpus]
    d_avg = float(sum(d)) / len(d)
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                f = float(idx[term][doc])
                score[doc] += i *  ( f / (float(d[doc]) / d_avg) )
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

In [56]:
results = get_results_tfidf_boost('zombie invasion', items_d)
print_results(results, 10)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook()
p = Scatter(df, x='score', y='length')
show(p)


Top 10 from recall set of 563 items:
	104.46 - reel zombies
	91.49 - zombie girl the movie
	86.68 - caustic zombies
	80.33 - zombie bloodbath
	75.57 - mathrubhoomi
	75.57 - gladiatress
	71.60 - first platoon
	68.70 - feeders
	60.99 - dead roses
	59.27 - time runner


Loading BokehJS ...

NameError: name 'Scatter' is not defined

## Implementing BM25

In [57]:
def get_results_bm25(qry, corpus, k1=1.5, b=0.75):
    idx = create_inverted_index(corpus)
    # 1.Assign (integer) n to be the number of documents in the corpus
    n = len(corpus)
    # 2.Assign (list) d with elements corresponding to the number of terms in each document in the corpus
    d = [len(x.split()) for x in corpus]
    # 3.Assign (float) d_avg as the average document length of the documents in the corpus
    d_avg = float(sum(d)) / len(d)                
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                # 4.Assign (float) f equal to the number of times the term appears in doc
                f = float(idx[term][doc])
                # 5.Assign (float) s the BM25 score for this (term, document) pair
                s = i * (( f * (k1 + 1) ) / (f + k1 * (1 - b + (b * (float(d[doc]) / d_avg)))))
                score[doc] += s
                
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

In [60]:
results = get_results_bm25('zombie survival night undead', items_d)
print_results(results, 10)


Top 10 from recall set of 7849 items:
	22.03 - zombie night awakening
	20.11 - the book of zombie
	19.95 - monster brawl
	19.71 - plane dead
	19.71 - zombie bloodbath
	19.51 - reel zombies
	19.48 - hood of the living dead
	18.12 - zombie vs mardi gras
	17.88 - the quick and the undead
	17.38 - children of the living dead


In [61]:
results = get_results_bm25('mountain bike', items_t, k1=1.5, b=0.75)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})
output_notebook()
p = Scatter(df, x='score', y='length')
show(p)

Loading BokehJS ...

NameError: name 'Scatter' is not defined